## Setup

In [1]:
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf

In [2]:
def shuffle(a, b, seed):
    'Shuffle two array in the same random order'
    rand_state = np.random.RandomState(seed)
    rand_state.shuffle(a)
    rand_state.seed(seed)
    rand_state.shuffle(b)

In [3]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32),
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/ID' + str(ID) + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [4]:
dummy_data = np.zeros((46, 501, 497, 13))
sample_segment_dim = (15,15)

# Calculate number of samples
sample_number_per_column = dummy_data.shape[1] - sample_segment_dim[0] + 1
sample_number_per_row = dummy_data.shape[2] - sample_segment_dim[1] + 1
#sample_number = dummy_data.shape[0] * sample_number_per_column * sample_number_per_row
sample_number = 100
print(sample_number)

100


In [5]:
ID_list = np.arange(sample_number)
labels = np.zeros(sample_number)

In [6]:
# Creating sample patches from the big images
# Change this code to snip samples from the images!
dummy_sample = np.zeros((15,15,13))
for x in range(0, sample_number):
    np.save('data/ID' + str(ID_list[x]) + '.npy', dummy_sample)

In [7]:
# Creating the Training and Validation sets
shuffle(ID_list, labels, 12045)

validation_split = 0.2
split_index = int((1-validation_split) * sample_number)

train_ID_list = ID_list[:split_index]
validation_ID_list = ID_list[split_index:]

In [8]:
# Create NN model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(4, (3,3), strides=(1, 1), input_shape=(15,15,13)))
print(model.output_shape)
model.add(tf.keras.layers.Flatten())
print(model.output_shape)
model.add(tf.keras.layers.Dense(16))
print(model.output_shape)
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

(None, 13, 13, 4)
(None, 676)
(None, 16)


In [9]:
# Initialize generators
params = {'dim': (15,15,13),
          'batch_size': 21,
          'n_classes': 3,
          'shuffle': True}
training_generator = DataGenerator(train_ID_list, labels, **params)
validation_generator = DataGenerator(validation_ID_list, labels, **params)

In [10]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    epochs=3,
                    validation_data=validation_generator)

Epoch 1/3
3/3 [==============================] - 0s 117ms/step - loss: 1.0912
Epoch 2/3
3/3 [==============================] - 0s 64ms/step - loss: 1.0554
Epoch 3/3
3/3 [==============================] - 0s 69ms/step - loss: 0.9895
